# Code for doing simulations
Here I upload all the necessary stuff

In [4]:
import numpy as np
from simulated_dipole import *
import matplotlib.pyplot as plt
import pandas as pd

keys_to_extract = ['dnn_xmax', 'energy', 'mass']

# IN CASE OF AixNet
# input_path = '/mnt/c/Users/paolo/Desktop/LAVORO/data_files/aixnet_sib_all_data.npz'
# data_all = np.load(input_path, allow_pickle=True)
# ddd = {key: data_all[key] for key in data_all.files}


## IN CASE OF KAne
input_kane = '../berenika_dipole/dateset/KAne_EPOS_sims.csv'

kf = pd.read_csv(input_kane)
ddd = {}
ddd['dnn_xmax'] = np.array(kf['Xmax'])
ddd['zenith'] = np.array(np.pi/2 - kf['Zenith'])
ddd['energy'] = np.array(kf['Energy']/1e18)
ddd['primary'] = np.array(kf['Primary'])
ddd['mass'] = np.empty_like(ddd['energy'])
ddd['mass'][np.where(ddd['primary']==0)] = 1.
ddd['mass'][np.where(ddd['primary']==1)] = 4.
ddd['mass'][np.where(ddd['primary']==2)] = 16.
ddd['mass'][np.where(ddd['primary']==3)] = 56.



print(ddd.keys())
print('Number of simulated events:', len(ddd['energy']))

n_samples = 1000
np.random.seed(90)
seed_1 = np.random.randint(1000, size=n_samples)
seed_2 = np.random.randint(1000, size=n_samples)
seed_3 = np.random.randint(1000, size=n_samples)

print('NAns:', np.isnan(ddd['dnn_xmax']).sum())

dict_keys(['dnn_xmax', 'zenith', 'energy', 'primary', 'mass'])
Number of simulated events: 808809
NAns: 2


In [5]:
def remove_nan_entries(d):
    # Create a mask of valid (non-NaN) entries based on one key (e.g., the first one)
    keys = list(d.keys())
    if not keys:
        return d  # empty dict
    
    # Start with all entries being valid
    mask = ~np.isnan(d[keys[0]])

    # Combine masks if any other key has NaNs
    for key in keys[1:]:
        mask &= ~np.isnan(d[key])

    # Apply mask to all entries
    return {key: np.array(d[key])[mask] for key in d}

ddd = {kk: ddd[kk] for kk in keys_to_extract if kk in ddd}
ddd = remove_nan_entries(ddd)
print(len(ddd['energy']))

808807


## Data production cycle

In [6]:
## classes I need (AixNet)
# mf = mass_fractions()
# es = energy_spectrum()
# smd = SMD_method()

## IN CASE OF KAne
mf = mass_fractions(model='EPOS')
es = energy_spectrum()
smd = SMD_method(model='EPOS')


# final results dictionary
final_results = {'iter': [], 'emin': [], 'light': [], 'heavy': [], 'frac_l': [], 'frac_h': []}

for k in range(n_samples):

    print('Iteration:', k+1, ' of ', n_samples)
    # data_mask = flatten_distr(ddd['energy'], seed=seed_1[k]) # only for AixNet, but not necessary
    # dd = dict_cutter(ddd, data_mask)

    dd = ddd
    print('After the first cut:', len(dd['energy']))

    # extract mass fractions
    mf.seed = seed_2[k]
    mass_mask = mf.extract_all_fractions(np.log10(dd['energy']*1e18), dd['mass'])
    dd = dict_cutter(dd, mass_mask)
    print('After mass fractions:', len(dd['energy']))

    # extract spectrum
    es.seed = seed_3[k]

    temp_dict = [{} for i in range(4)]

    for i in range(len(energies[:-1])):
        enne = energies[i]/1e18
        enneplus = energies[i+1]/1e18
        energy_mask = (dd['energy']>=enne)&(dd['energy']<enneplus)
        temp_dict[i] = dict_cutter(dd, energy_mask)
        spectrum_mask = es.spectrum_fraction(temp_dict[i]['energy']*1e18)
        temp_dict[i] = dict_cutter(temp_dict[i], spectrum_mask)

    dd = dict_paster(temp_dict)
    print('After energy spectrum:', len(dd['energy']))

    # define charge
    dd['charge'] = np.empty_like(dd['mass'])
    for mass in names:
        dd['charge'][np.where(dd['mass']==names[mass])]=charges[mass]

    generate_xmax19(dd, 'dnn_xmax', 'energy')
    print('NAns:', np.isnan(dd['dnn_xmax']).sum())

    # plt.figure(figsize=(10,6))
    # #plt.title('Simulated data')
    # plt.scatter(dd['energy']*1e18, dd['xmax19'], alpha=0.3, color='k')
    # plt.xlabel(r'energy / eV')
    # plt.ylabel(r'$X_\mathrm{max} / \mathrm{(g \, cm^{-2})}$')
    # plt.xscale('log')
    # plt.ylim(600, 1200)
    # plt.xlim(2.5e18, 150e18)
    # plt.show()
    
    # estimate best quantiles
    thres_values = np.linspace(np.min(dd['xmax19']), np.max(dd['xmax19']), 27)
    # print(thres_values)
    res = { 'emin': [], 
            'light': [], 
            'heavy': [], 
            'N_light': [],
            'N_heavy': [],
            'N_tot': [],
            'SMD': [],
            }
    
    for ee in range(len(energies)-1):
        e_min = energies[ee]/1e18
        e_max =energies[ee+1]/1e18


        # extract energy range
        e_mask = (dd['energy']<e_max)&(dd['energy']>=e_min)
        data = dict_cutter(dd, e_mask)

        for i in range(1, len(thres_values) - 2):
            thres_h = thres_values[i]

            for j in range(i + 1, len(thres_values) - 1):

                thres_l = thres_values[j]
                mask_h = (data['xmax19']<thres_h)
                mask_l = (data['xmax19']>thres_l)
                light_frac = dict_cutter(data, mask_l)
                heavy_frac = dict_cutter(data, mask_h)

                value = smd.quantify_SMD(light_frac['energy'], light_frac['charge'], heavy_frac['energy'], heavy_frac['charge'], d_max=1)
                A_l = np.sum(np.log(light_frac['mass']))
                A_h = np.sum(np.log(heavy_frac['mass']))

                res['emin'].append(e_min)
                res['light'].append(thres_l)
                res['heavy'].append(thres_h)
                res['SMD'].append(value)
                res['N_light'].append(len(light_frac['xmax19']))
                res['N_heavy'].append(len(heavy_frac['xmax19']))
                res['N_tot'].append(len(data['xmax19']))

    for key, value in res.items():
        res[key] = np.array(value)

    # format final results
    for l in range(len(energies[:-1])):
        emin_val = energies[l] / 1e18
        mask = res['emin'] == emin_val

        if not np.any(mask):
            print(f"[WARNING] No SMD entries found for energy bin {emin_val:.2f} EeV (iteration {k})")
            continue

        smd_vals = res['SMD'][mask]
        if len(smd_vals) == 0 or np.all(np.isnan(smd_vals)):
            print(f"[WARNING] SMD array is empty or NaN for energy bin {emin_val:.2f} EeV (iteration {k})")
            continue

        max_pos = np.argmax(smd_vals)
        valid_indices = np.where(mask)[0]
        if len(valid_indices) == 0:
            print(f"[WARNING] No valid indices found after masking for {emin_val:.2f} EeV (iteration {k})")
            continue

        idx = valid_indices[max_pos]

        n_tot = res['N_tot'][idx]
        n_light = res['N_light'][idx]
        n_heavy = res['N_heavy'][idx]

        if n_tot == 0:
            print(f"[WARNING] Total events is zero for energy bin {emin_val:.2f} EeV (iteration {k})")
            frac_l = np.nan
            frac_h = np.nan
        else:
            frac_l = n_light / n_tot
            frac_h = n_heavy / n_tot

        final_results['iter'].append(k)
        final_results['emin'].append(emin_val)
        final_results['light'].append(res['light'][idx])
        final_results['heavy'].append(res['heavy'][idx])
        final_results['frac_l'].append(frac_l)
        final_results['frac_h'].append(frac_h)




# Convert to pandas DataFrame
df_results = pd.DataFrame(final_results)
display(df_results)
df_results.to_csv("final_results_kane.csv", index=False)

Iteration: 1  of  1000
After the first cut: 808807


After mass fractions: 198124
After energy spectrum: 49540
NAns: 0
Iteration: 2  of  1000
After the first cut: 808807
After mass fractions: 198432
After energy spectrum: 49341
NAns: 0
Iteration: 3  of  1000
After the first cut: 808807
After mass fractions: 198902
After energy spectrum: 49788
NAns: 0
Iteration: 4  of  1000
After the first cut: 808807
After mass fractions: 198509
After energy spectrum: 49799
NAns: 0
Iteration: 5  of  1000
After the first cut: 808807
After mass fractions: 198924
After energy spectrum: 49675
NAns: 0
Iteration: 6  of  1000
After the first cut: 808807
After mass fractions: 198459
After energy spectrum: 49386
NAns: 0
Iteration: 7  of  1000
After the first cut: 808807
After mass fractions: 198392
After energy spectrum: 49552
NAns: 0
Iteration: 8  of  1000
After the first cut: 808807
After mass fractions: 198883
After energy spectrum: 49632
NAns: 0
Iteration: 9  of  1000
After the first cut: 808807
After mass fractions: 198371
After energy spectrum: 49548
NAns: 

,iter,emin,light,heavy,frac_l,frac_h
0,0,4.0,797.192914,762.470979,0.321042,0.397759
1,0,8.0,797.192914,745.110011,0.218655,0.336831
2,0,16.0,797.192914,745.110011,0.133087,0.447288
3,0,32.0,779.831946,727.749044,0.114168,0.430016
4,1,4.0,813.073862,743.689089,0.218548,0.240137
...,...,...,...,...,...,...
3995,998,32.0,787.355935,732.539054,0.084569,0.485993
3996,999,4.0,813.216820,759.101261,0.224105,0.365247
3997,999,8.0,813.216820,741.062742,0.139950,0.292094
3998,999,16.0,795.178300,741.062742,0.140285,0.403386
